# Fully 3D Geometry Visualisation

Authors: David Atkinson

First version: 20 June 2021

CCP SyneRBI Synergistic Image Reconstruction Framework (SIRF).
Copyright 2021 University College London.

This is software developed for the Collaborative Computational Project in Synergistic Reconstruction for Biomedical Imaging (http://www.ccpsynerbi.ac.uk/).
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This Notebook aims to help users check the orientation of their data by visualising it in 3D. It plots intersecting orthogonal MR slices and a PET slice from a phantom in a PET/MR scanner, and a separate phantom from Philips MR.

Using the affine matrix in SIRF's geometry we can calculate the 3D LPH+ patient position of each pixel and plot slices in 3D. The plot can be rotated using a mouse.

In [ ]:
# Set data folder location
# See notebook get_nifti_data  to download data
data_path = '/devel'

In [ ]:
%matplotlib notebook

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import sirf.Reg as Reg

In [ ]:
def sdisp(ax, s_ido, falpha, frame, ocmap='gray'):
    # 3D Display of SIRF ImageData Object
    # sdisp(ax, s_ido, falpha, frame, ocmap='gray')
    #  ax      axes predefined
    #  s_ido   SIRF ImageData Object
    #  falpha  face alpha (0-1)
    #  frame   frame number (0-based)
    #  ocmap   colormap defaults to gray
    #
    # Calculates the vertices of pixels and uses to create a surface with transparency
    # falpha and intensities corresponding to pixwl values
    
    s_geoinfo = s_ido.get_geometrical_info()
    s_array   = s_ido.as_array() 
    
    img = s_array[:,:,frame]
    
    nrow = img.shape[0]
    ncol = img.shape[1]
    
    L = np.zeros((nrow+1, ncol+1))  # allocate memory
    P = np.zeros((nrow+1, ncol+1))  # +1 because this is for vertices
    H = np.zeros((nrow+1, ncol+1))
    
    A = s_geoinfo.get_index_to_physical_point_matrix() 
    
    for ir in range(0,nrow+1):
        for ic in range(0,ncol+1):
            # VLPH are LPH patient coordinates corresponding to
            # pixel vertices, which are at image coords -0.5, 0.5, 1.5, ...
            VLPH = np.matmul(A, np.array([ [ir-0.5], [ic-0.5], [frame], [1] ]))
            
            L[ir,ic] = VLPH[0] #  separate the components for surf plot
            P[ir,ic] = VLPH[1] 
            H[ir,ic] = VLPH[2] 
    
    scamap = plt.cm.ScalarMappable(cmap=ocmap)
    fcolors = scamap.to_rgba(img, alpha=falpha)
    ax.plot_surface(L, P, H, facecolors=fcolors, cmap=ocmap, linewidth=0, rcount=100, ccount=100)
    ax.set_xlabel('Left')
    ax.set_ylabel('Posterior')
    ax.set_zlabel('Head')

In [ ]:
def getido(fn, fpath):
    # returns a SIRF ImageData Object, given filename and path
    ffn = os.path.join(fpath, fn)
    s_ido = Reg.ImageData(ffn)  # load file into SIRF ImageData object
    return s_ido

In [ ]:
fig = plt.figure()          # Open figure and get 3D axes (can rotate with mouse)
ax  = plt.axes(projection='3d') 

fpath  = os.path.join(data_path , 'nifti')

# Data from mMR Biograph. DICOM data converted to NIfTI using MRIcroGL which
# uses dcm2niix

#mr_cor_ido = getido("t2_tse_cor_20180822174706_5.nii", fpath)
#sdisp(ax, mr_cor_ido, 0.4, 15)

mr_sag_ido = getido("t2_tse_sag_20180822174706_3.nii", fpath)
sdisp(ax, mr_sag_ido, 0.7, 12)

mr_tra_ido = getido("t2_tse_axi_20180822174706_4.nii", fpath)
sdisp(ax, mr_tra_ido, 0.7, 15)

pet_ido = getido("Head_MRAC_PET_UTE_20180822174706_7.nii", fpath)
sdisp(ax, pet_ido, 0.8, 63, 'hot')


There should be a 3D figure above showing PET tracer and MR slices. It can be rotated using the mouse, though can be a bit laggy.

In [ ]:
fig = plt.figure()          # Open figure and get 3D axes (can rotate with mouse)
ax  = plt.axes(projection='3d') 

fpath  = os.path.join(data_path, 'nifti')

# Data from Philips MR
cor_ido = getido("OBJECT_phantom_T2W_TSE_Cor_14_1.nii", fpath)   #use frame 15
#obl_ido = getido("OBJECT_phantom_T2W_TSE_OBL_19_1.nii", fpath)  #frame 19
tra_ido = getido("OBJECT_phantom_T2W_TSE_Tra_17_1.nii", fpath)   #frame 2
#sag_ido = getido("OBJECT_phantom_T2W_TSE_Sag_18_1.nii", fpath)

sdisp(ax, cor_ido, 0.4, 15)
sdisp(ax, tra_ido, 0.4,  2)


If the geometry information is correct, the slice intersections should be correct and the axes should point in the correct directions (take care to observe the positive direction on the axes). Note matplotlib 3D don't seem to support equal aspect ratios in 3D so they can be deceptive  - look at the axis number labels.
Because the plots all use the patient coordinate system, they are correct, even for different orientations, fields of view and pixel sizes.